In [1]:
import os
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
# import cv2
import pandas as pd
from PIL import Image, ImageFilter

import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

import tensorflow as tf

Using TensorFlow backend.


In [2]:
# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend" #for GPU Support on MacBook
print(tf.__version__)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

1.13.1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4482970525919741599
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4945621811
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5193168271277820073
physical_device_desc: "device: 0, name: GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [4]:
currentDir = os.getcwd()
datasetDir = currentDir + "\\FilteredImages\\"
# datasetDir = currentDir + "/FilteredImages2/" # NEW SINGLE IMAGES DIRECTORY
trainDir = os.path.join(datasetDir, "train")
testDir = os.path.join(datasetDir, "test")
validDir = os.path.join(datasetDir, "valid")
y_dataDir = os.path.join(datasetDir, "y_data")

In [5]:
def sortFirst(val):
    return val[0]

def getBinary(dataFile):
    listOfTraining = []
    trainingHeader = []
    with open(dataFile) as csvfile:
#     reader = csv.DictReader(csvfile)
        reader2 = csv.reader(csvfile)
        listOfTraining = list(reader2)
        trainingHeader = listOfTraining[0]
        listOfTraining.pop(0)
#         listOfTraining = listOfTraining.sort(key = sortFirst, reverse = False)
#         np.asarray(listofTraining, dtype=np.int32)
#         return np.asarray(listofTraining, dtype=np.int32)
    listOfTrainingBinary = []
    for item in listOfTraining:
        listOfTrainingBinary.append(item[1])
    return np.asarray(listOfTrainingBinary, dtype=np.int32)

In [6]:
y_train = []
y_trainDir = os.path.join(y_dataDir, 'train_split_Depression_AVEC2017-edited.csv')
# print(y_trainDir)
y_train = getBinary(y_trainDir)
y_testDir = os.path.join(y_dataDir, 'dev_split_Depression_AVEC2017.csv')
# print(y_testDir)
y_test = getBinary(y_testDir)
print(y_train)
print(y_test)

# Y_train = np.asarray(y_train, dtype=np.int32)

[0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1
 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0
 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0]


In [7]:
x_trainDir = trainDir
x_testDir = testDir

In [8]:
x_trainDir

'C:\\Users\\tanho\\Downloads\\Aaron-Workspace\\FilteredImages\\train'

In [9]:
# testSet = []
# testFolders =[]
# # trainingSet = []
# trainingFolders = []
# x_train = []
# y_train = []

def getImagesDir(mainDirectory):
    ImgDict = {}
    ImgList = []
    
    for folder in os.listdir(mainDirectory):
        theDir = os.path.join(mainDirectory, folder)
        images = os.listdir(theDir)
        listOfImgDir = []
        for img in images:
#             listOfImgDir.append(Image.open(os.path.join(theDir, img)))
            listOfImgDir.append(os.path.join(theDir, img))
#         print(listOfImgDir)
#         print(images)
        folderName = folder.split('_')
        ImgDict[folderName[0]] = listOfImgDir
        ImgList.append(listOfImgDir)
    imgList = ImgList.sort(key = sortFirst, reverse = False)
    return ImgList
        
        
#         for img in images
    
# for folder in toProcessList:
#     images = os.listdir(datasetDir + "/" + folder)
#     folderName = folder.split('_')
# #     print(folderName[0])
#     if folderName[0] in listOfTrainingName:
#         trainingFolders.append(datasetDir + "/" + folder)
# #         print(folderName)
#         index = listOfTrainingName.index(folderName[0])
#         temp = listOfTraining[index]
#         x_train.append(images)
#         tempBList = []
#         val = temp[1]
#         tempBList.append(val)
#         y_train.append(tempBList)
#         #temp.append(images)
#         #trainingSet.append(temp)
#     else:
#         testFolders.append(datasetDir + "/" + folder)
#         testSet.append(images)

trainingImagesDir = getImagesDir(trainDir)
testImagesDir = getImagesDir(testDir)
# np.array(trainingImages).shape

In [10]:
def importImages(listOfImgDir):
    myFolder = []
    for folder in listOfImgDir:
        myImages = []
        for image in folder:
            myImages.append(np.array(Image.open(image)))
        myFolder.append(np.array(myImages))
    return myFolder


#First Array iterate through Folder, Second Array Iterate though Image in Folder
trainingImages = importImages(trainingImagesDir)
testImages = importImages(testImagesDir)

In [11]:
#Num of Img, Height, Width
print(len(trainingImages)) 

len(y_train)

107


107

In [12]:
trainingImagesNP = np.array(trainingImages)
testImagesNP = np.array(testImages)

In [13]:
# print(trainingImages.shape)
# print(trainingImages[0])
# print(trainingImagesNP[10].shape)
# print(y_train)

neg = []
y_neg = []
pos = []
y_pos = []

for x,y in zip(trainingImages, y_train):
    if (y == 0):
        for each in x:
            neg.append(each)
            y_neg.append(0)
    else:
        for each in x:
            pos.append(each)
            y_pos.append(1)

X = pos + neg
Y = y_pos + y_neg

In [14]:
neg_test = []
y_neg_test = []
pos_test = []
y_pos_test = []

for x,y in zip(testImages, y_train):
    if (y == 0):
        for each in x:
            neg_test.append(each)
            y_neg_test.append(0)
    else:
        for each in x:
            pos_test.append(each)
            y_pos_test.append(1)

X_test = pos_test + neg_test
Y_test = y_pos_test + y_neg_test


npX_test = np.array(X_test)
npY_test = np.array(Y_test)
newNPX_test = npX_test.reshape(npX_test.shape[0], 101, 1000, 1)

In [15]:
vImagesDir = getImagesDir(validDir)
vImages = importImages(vImagesDir)
vImagesNP = np.array(vImages)

#TESTING DATA UNSEEN DATA
v = []
# y_neg_v = []
# pos_v = []
# y_pos_v = []

for x in vImages:
    for each in x:
        v.append(each)

# X_v = pos_v + neg_v
# Y_v = y_pos_v + y_neg_v


np_v = np.array(v)
# npY_test = np.array(Y_test)
newNPX_v = np_v.reshape(np_v.shape[0], 101, 1000, 1)

In [16]:
npX = np.array(X)
npY = np.array(Y)

In [17]:
print(npX.shape)
print(npY.shape)

(4591, 101, 1000)
(4591,)


In [18]:
newNPX = npX.reshape(npX.shape[0], 101, 1000, 1)

In [19]:
print(newNPX.shape)

(4591, 101, 1000, 1)


In [20]:
# print(trainingImagesNP)

In [21]:
# input_shape = (101, 1000, 3)

# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=input_shape))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation('relu'))
# # model.add(Dropout(0.5))
# model.add(Dense(1))
# model.add(Activation('sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# model.summary()

In [22]:
# from keras.preprocessing.image import ImageDataGenerator

In [23]:
#Saving Model
from keras.callbacks import ModelCheckpoint
#numEpochs = 100
numEpochs = 30
model_path = os.path.join(currentDir,'DAM-DHM-V-'+ str(numEpochs) +'.h5')

# checkpoint = ModelCheckpoint(model_path, monitor='loss', verbose=1, save_best_only=True, mode='max')

# checkpoint = ModelCheckpoint(model_path, monitor='acc', verbose=1, save_best_only=True, mode='max')

checkpoint = ModelCheckpoint(model_path, monitor='acc', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [checkpoint, tbCallBack]

In [24]:
# input_shape

In [25]:
input_shape = (101, 1000, 1)

In [26]:
# DR HARRY
from keras.callbacks import EarlyStopping

def evaluate_model(X_train, X_val, y_train, y_val):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(AveragePooling2D(pool_size=(4, 3)))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(AveragePooling2D(pool_size=(4, 3)))

    model.add(Conv2D(96, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(96, (3, 3), padding='same', activation='relu'))
    model.add(AveragePooling2D(pool_size=(4, 3)))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adadelta', #adam
                  metrics=['accuracy']) 
    
    print(model.metrics_names)
    
    model.save_weights('model.h5')
    
    callbacks = [EarlyStopping(monitor='val_acc', patience = 10)]
    
    hist=model.fit(x=X_train, y=y_train, batch_size=32, epochs=numEpochs, callbacks=callbacks_list, validation_data=(X_val, y_val))
    
    _, val_acc=model.evaluate(x=X_val, y=y_val, verbose=1)
  
    model.load_weights('model.h5')
    
    model.summary()
    
    print("acc: ", np.mean(hist.history['acc']))
    
    print("val_acc: ", val_acc)
    
    model.save(''+ str(numEpochs) +' Epoch (Dr Harry Model) w validation.h5')
    
    return model, val_acc

In [27]:
# run CNN model
model, val_acc = evaluate_model(newNPX, newNPX_test, npY, npY_test) 

Instructions for updating:
Colocations handled automatically by placer.
['loss', 'acc']
Instructions for updating:
Use tf.cast instead.
Train on 4591 samples, validate on 1740 samples
Epoch 1/30
4591/4591 [==============================] - 40s 9ms/step - loss: 0.6199 - acc: 0.7136 - val_loss: 0.8270 - val_acc: 0.5902

Epoch 00001: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 2/30
4591/4591 [==============================] - 35s 8ms/step - loss: 0.5999 - acc: 0.7188 - val_loss: 0.7076 - val_acc: 0.5902

Epoch 00002: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 3/30
4591/4591 [==============================] - 35s 8ms/step - loss: 0.5951 - acc: 0.7188 - val_loss: 0.7179 - val_acc: 0.5902

Epoch 00003: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 4/30
4591/4591 [==============================] - 35s 8ms/step - loss: 0.5957 - acc: 0.7188 - val_loss: 0.7425 - val_acc: 0.5902

Epoch 00004: s

In [28]:
# k-fold cross validation, k = 10

n_folds = 10
count = 0
cv_scores, model_history = list(), list()
for _ in range(n_folds):
    # split data
    X_train, X_val, y_train, y_val = train_test_split(newNPX, npY, test_size=0.10, random_state = np.random.randint(1,1000, 1)[0])
    # evaluate model
    model, test_acc = evaluate_model(X_train, X_val, y_train, y_val)
    count += 1
    cv_scores.append(test_acc)
    model_history.append(model)
    print('K-Fold has ran ', count, ' times')
    
print('\nModel Accuracy after all K-Fold: ', (np.mean(cv_scores))

['loss', 'acc']
Train on 4131 samples, validate on 460 samples
Epoch 1/30
4131/4131 [==============================] - 30s 7ms/step - loss: 4.1850 - acc: 0.7139 - val_loss: 0.6859 - val_acc: 0.6717

Epoch 00001: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 2/30
4131/4131 [==============================] - 29s 7ms/step - loss: 0.6081 - acc: 0.7240 - val_loss: 0.6400 - val_acc: 0.6717

Epoch 00002: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 3/30
4131/4131 [==============================] - 29s 7ms/step - loss: 0.5932 - acc: 0.7240 - val_loss: 0.6680 - val_acc: 0.6717

Epoch 00003: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 4/30
4131/4131 [==============================] - 29s 7ms/step - loss: 0.5928 - acc: 0.7206 - val_loss: 0.6394 - val_acc: 0.6717

Epoch 00004: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 5/30
4131/4131 [===========================

['loss', 'acc']
Train on 4131 samples, validate on 460 samples
Epoch 1/30
4131/4131 [==============================] - 29s 7ms/step - loss: 0.6442 - acc: 0.7105 - val_loss: 0.6227 - val_acc: 0.7109

Epoch 00001: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 2/30
4131/4131 [==============================] - 29s 7ms/step - loss: 0.6053 - acc: 0.7197 - val_loss: 0.6011 - val_acc: 0.7109

Epoch 00002: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 3/30
4131/4131 [==============================] - 29s 7ms/step - loss: 0.5962 - acc: 0.7197 - val_loss: 0.6017 - val_acc: 0.7109

Epoch 00003: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 4/30
4131/4131 [==============================] - 29s 7ms/step - loss: 0.5955 - acc: 0.7197 - val_loss: 0.6007 - val_acc: 0.7109

Epoch 00004: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 5/30
4131/4131 [===========================

['loss', 'acc']
Train on 4131 samples, validate on 460 samples
Epoch 1/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6368 - acc: 0.7112 - val_loss: 0.9010 - val_acc: 0.6978

Epoch 00001: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 2/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5968 - acc: 0.7211 - val_loss: 0.6209 - val_acc: 0.6978

Epoch 00002: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 3/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5949 - acc: 0.7211 - val_loss: 0.6130 - val_acc: 0.6978

Epoch 00003: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 4/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5963 - acc: 0.7211 - val_loss: 0.6203 - val_acc: 0.6978

Epoch 00004: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 5/30
4131/4131 [===========================

['loss', 'acc']
Train on 4131 samples, validate on 460 samples
Epoch 1/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6302 - acc: 0.7160 - val_loss: 0.6285 - val_acc: 0.7130

Epoch 00001: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 2/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5989 - acc: 0.7194 - val_loss: 0.6071 - val_acc: 0.7130

Epoch 00002: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 3/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5985 - acc: 0.7194 - val_loss: 0.6109 - val_acc: 0.7130

Epoch 00003: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 4/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5968 - acc: 0.7194 - val_loss: 0.6002 - val_acc: 0.7130

Epoch 00004: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 5/30
4131/4131 [===========================

['loss', 'acc']
Train on 4131 samples, validate on 460 samples
Epoch 1/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.9047 - acc: 0.7146 - val_loss: 0.6063 - val_acc: 0.7043

Epoch 00001: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 2/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5983 - acc: 0.7204 - val_loss: 0.6144 - val_acc: 0.7043

Epoch 00002: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 3/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5946 - acc: 0.7204 - val_loss: 0.6108 - val_acc: 0.7043

Epoch 00003: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 4/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5941 - acc: 0.7204 - val_loss: 0.6073 - val_acc: 0.7043

Epoch 00004: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 5/30
4131/4131 [===========================

['loss', 'acc']
Train on 4131 samples, validate on 460 samples
Epoch 1/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.7312 - acc: 0.7107 - val_loss: 0.7293 - val_acc: 0.7000

Epoch 00001: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 2/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5969 - acc: 0.7209 - val_loss: 0.7057 - val_acc: 0.7000

Epoch 00002: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 3/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5949 - acc: 0.7209 - val_loss: 0.6160 - val_acc: 0.7000

Epoch 00003: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 4/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5953 - acc: 0.7209 - val_loss: 0.6194 - val_acc: 0.7000

Epoch 00004: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 5/30
4131/4131 [===========================

['loss', 'acc']
Train on 4131 samples, validate on 460 samples
Epoch 1/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6617 - acc: 0.7182 - val_loss: 0.5946 - val_acc: 0.7196

Epoch 00001: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 2/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5997 - acc: 0.7187 - val_loss: 0.5945 - val_acc: 0.7196

Epoch 00002: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 3/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6223 - acc: 0.7187 - val_loss: 0.5931 - val_acc: 0.7196

Epoch 00003: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 4/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5963 - acc: 0.7187 - val_loss: 0.5947 - val_acc: 0.7196

Epoch 00004: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 5/30
4131/4131 [===========================

['loss', 'acc']
Train on 4131 samples, validate on 460 samples
Epoch 1/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6443 - acc: 0.7088 - val_loss: 0.5869 - val_acc: 0.7522

Epoch 00001: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 2/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6062 - acc: 0.7151 - val_loss: 0.5713 - val_acc: 0.7522

Epoch 00002: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 3/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6003 - acc: 0.7151 - val_loss: 0.5694 - val_acc: 0.7522

Epoch 00003: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 4/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6005 - acc: 0.7151 - val_loss: 0.5674 - val_acc: 0.7522

Epoch 00004: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 5/30
4131/4131 [===========================

['loss', 'acc']
Train on 4131 samples, validate on 460 samples
Epoch 1/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6291 - acc: 0.7122 - val_loss: 1.0347 - val_acc: 0.7000

Epoch 00001: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 2/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6001 - acc: 0.7209 - val_loss: 0.6175 - val_acc: 0.7000

Epoch 00002: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 3/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6017 - acc: 0.7209 - val_loss: 0.6104 - val_acc: 0.7000

Epoch 00003: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 4/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5944 - acc: 0.7199 - val_loss: 0.6126 - val_acc: 0.7000

Epoch 00004: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 5/30
4131/4131 [===========================

['loss', 'acc']
Train on 4131 samples, validate on 460 samples
Epoch 1/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6406 - acc: 0.7134 - val_loss: 0.6301 - val_acc: 0.6978

Epoch 00001: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 2/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6012 - acc: 0.7211 - val_loss: 0.6141 - val_acc: 0.6978

Epoch 00002: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 3/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.6046 - acc: 0.7211 - val_loss: 0.6126 - val_acc: 0.6978

Epoch 00003: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 4/30
4131/4131 [==============================] - 28s 7ms/step - loss: 0.5939 - acc: 0.7211 - val_loss: 0.6124 - val_acc: 0.6978

Epoch 00004: saving model to C:\Users\tanho\Downloads\Aaron-Workspace\DAM-DHM-V-30.h5
Epoch 5/30
4131/4131 [===========================

In [29]:
result = model.predict(newNPX_v)
for each in result:
    print(each)

[0.67006874]
[0.6867978]
[0.6458308]
[0.6805024]
[0.7335013]
[0.67324644]
[0.74081373]
[0.7228327]
[0.67381525]
[0.65047807]
[0.6291201]
[0.65658724]
[0.6722251]
[0.6627605]
[0.69858]
[0.6795434]
[0.6168731]
[0.68014544]
[0.654198]
[0.68004155]
[0.681952]
[0.65644014]
[0.6521288]
[0.654008]
[0.6328309]
[0.6890311]
[0.6559533]
[0.6681738]
[0.67244244]
[0.64830637]
[0.7148792]
[0.6750471]
[0.68882346]
[0.66350317]
[0.6406661]
[0.64942265]
[0.67649937]
[0.71295863]
[0.7029922]
[0.65047467]
[0.64842266]
[0.7030382]
[0.6649402]
[0.6594435]
[0.6526973]
[0.6450184]
[0.6911686]
[0.6500969]
[0.6386665]
[0.6863981]
[0.6683552]
[0.6907112]
[0.6612068]
[0.65292215]
[0.6948359]
[0.64810765]
[0.66757846]
[0.642527]
[0.6308431]
[0.6619879]
[0.6717275]
[0.66501456]
[0.68367374]
[0.660515]
[0.6717639]
[0.67846674]
[0.7058482]
[0.6940516]
[0.66579384]
[0.6841956]
[0.6617938]
[0.69447774]
[0.6811924]
[0.65719223]
[0.6900504]
[0.67798954]
[0.6778217]
[0.68577707]
[0.68008953]
[0.6703056]
[0.6892375]
[0.69

[0.6208962]
[0.6170454]
[0.60704255]
[0.597344]
[0.61931145]
[0.59160304]
[0.6623666]
[0.63145864]
[0.60982025]
[0.65798026]
[0.6264495]
[0.62035525]
[0.59073335]
[0.5814898]
[0.5991826]
[0.6522672]
[0.6068334]
[0.6269746]
[0.6262111]
[0.61254287]
[0.5764158]
[0.63641554]
[0.613021]
[0.60051894]
[0.615793]
[0.6085661]
[0.5839881]
[0.6441444]
[0.6346591]
[0.64251554]
[0.63197684]
[0.6229242]
[0.6468233]
[0.63154864]
[0.6438778]
[0.6289761]
[0.6234197]
[0.6001253]
[0.63057894]
[0.63669306]
[0.6624966]
[0.6381746]
[0.64141953]
[0.6225457]
[0.6097769]
[0.60223126]
[0.63513285]
[0.59679556]
[0.6025806]
[0.63161767]
[0.5983059]
[0.59819555]
[0.6148097]
[0.63068616]
[0.6397841]
[0.5934433]
[0.6608938]
[0.6252583]
[0.6230444]
[0.60544497]
[0.61641425]
[0.57900536]
[0.6077384]
[0.62012506]
[0.63010675]
[0.6062742]
[0.5848024]
[0.6398063]
[0.6065512]
[0.6067162]
[0.6222962]
[0.5979254]
[0.5586511]
[0.6163823]
[0.5795903]
[0.6156366]
[0.61432195]
[0.62549555]
[0.6309551]
[0.5969281]
[0.6281186]
[

In [36]:
np.mean(cv_scores)

0.7019565217909605